<a href="https://colab.research.google.com/github/Yuval-Br/YuvRep/blob/main/Pytorch_basics.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [4]:
import numpy as np
import torch
import torch.nn as nn
from torch.utils.data import DataLoader
from torchvision import datasets, transforms
from torchvision.utils import make_grid
import torch.nn.functional as f
import matplotlib.pyplot as plt
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix



Tensors - The Pytorch array

In [ ]:
tensor_2d = torch.randn(3,4)

tensor_3d = torch.randn(2,4,3)  # matrices, rows in each, cols in each

a = np.array([2,3,4])
at = torch.tensor(a) # convert np to tensor

t1 = torch.tensor([[1,2,3],[1,4,2]]) # from a list


Basic Tensor Operations

In [ ]:
# shaping
te = torch.arange(10)
te1 = te.reshape(5,2) # reshape like np
te2 = te.reshape(2,-1) # reshape into a 2 row matrix without knowing number elements, using -1.
# changes to te affect te1, te2.

In [ ]:
# slicing
te[6] # returns a tensor
tensor_2d[:,2] # all rows the 2 col, in a 1d tensor
tensor_3d[1,1:4,1] # first matrix, 1 to 3 rows, 1 col.

tensor([-1.0032, -0.7666,  0.1413])

In [ ]:
ta = torch.tensor([1,2,3,4])
tb = torch.tensor([1,2,4,6])
ta + tb , torch.add(ta,tb) # addition, in 2 ways.
ta - tb, torch.sub(ta,tb) # subtraction
ta*tb, torch.mul(ta,tb) # element multiply
ta/tb, torch.div(ta,tb) # div
ta % tb, torch.remainder(ta,tb) # ta mod tb, elementwise.
torch.pow(ta,tb) # ta^tb, element wise.
ta.pow(tb) # same

# broadcasting like np:
ta.pow(2)
torch.tensor([1,2,3]) + torch.tensor([[0,0,0],[1,2,3]])

tensor([[1, 2, 3],
        [2, 4, 6]])

Basic Neural Network

In [ ]:
# creating the class for a specific model (for iris data set(n=4, Labels=3)):
class Model(nn.Module):
  # Input layer (4 features) -->FC
  # Hidden layer 1 (8 neurons) -->relu,FC
  # Hidden layer 2 (8 neurons) -->relu,FC
  # Output layer (3 labels)
  def __init__(self,in_features=4,h1=8,h2=8, out_features=3):
    super().__init__()
    self.fc1 = nn.Linear(in_features,h1)
    self.fc2 = nn.Linear(h1,h2)
    self.out = nn.Linear(h2,out_features)
  def forward(self,x):
    x = f.relu(self.fc1(x))
    x = f.relu(self.fc2(x))
    x = self.out(x)

    return x



In [ ]:
model = Model()

In [ ]:
# Loading, Preparing the Data
iris = pd.read_csv('https://gist.githubusercontent.com/curran/a08a1080b88344b0c8a7/raw/0e7a9b0a5d22642a06d3d5b9bcbad9890c8ee534/iris.csv')
iris['species'] = iris['species'].replace('setosa', 0)
iris['species'] = iris['species'].replace('versicolor', 1)
iris['species'] = iris['species'].replace('virginica', 2)
x = iris.drop(columns='species').values
y = iris['species'].values

In [ ]:
# Splitting Data, converting to tensors.
x_train,x_test,y_train,y_test = train_test_split(x,y,test_size=0.1)
x_train = torch.FloatTensor(x_train)
x_test = torch.FloatTensor(x_test)
y_train = torch.LongTensor(y_train)
y_test = torch.LongTensor(y_test)

In [ ]:
criterion = nn.CrossEntropyLoss()  # loss function.
optimizer = torch.optim.Adam(model.parameters(), lr=0.01)  # optimizer and learning rate.

In [ ]:
# training
epochs = 100  # in each epoch we run through the data (SGD)
losses = [] # the loss in each epoch
for i in range(epochs):
  y_pred = model.forward(x_train) # current model prediction
  loss = criterion(y_pred,y_train) # the loss
  losses.append(loss.detach().numpy()) # add to the list
  optimizer.zero_grad() # Backprop
  loss.backward()
  optimizer.step()



In [ ]:
plt.plot(range(epochs),losses) # plott the loss decrease in epochs

In [ ]:
# test data eval:
with torch.no_grad(): # turn off backprop
  y_eval = model.forward(x_test)
  loss_test = criterion(y_eval,y_test)

loss_test



In [ ]:
# count how much we got right from test set
correct= 0
with torch.no_grad():
  for i, data in enumerate(x_test):
    y_val = model.forward(data)
    if y_val.argmax().item() == y_test[i]:
      correct += 1

correct


In [ ]:
# evaluating on new data
new = torch.tensor([4.7,3.2,1.3,0.2])
print(model(new)) # largest value is the pred label.


In [ ]:
# save the model
torch.save(model.state_dict(), 'iris_model.pt')

In [ ]:
# load the model
new_model = Model() # same class of model for iris.
new_model.load_state_dict(torch.load('iris_model.pt')) # loading the weights from training.

<All keys matched successfully>